# Bayesian global optimization with gaussian processes for finding (sub-)optimal parameters of LightGBM

As many of fellow kaggler asking how did I get LightGBM parameters for the kernel [Customer Transaction Prediction](https://www.kaggle.com/fayzur/customer-transaction-prediction) I published. So, I decided to publish a kernel to optimize parameters. 



In this kernel I use Bayesian global optimization with gaussian processes for finding optimal parameters. This optimization attempts to find the maximum value of an black box function in as few iterations as possible. In our case the black box function will be a function that I will write to optimize (maximize) the evaluation function (AUC) so that parameters get maximize AUC in training and validation, and expect to do good in the private. The final prediction will be **rank average on 5 fold cross validation predictions**.

Continue to the end of this kernel and **upvote it if you find it is interesting**.

![image.jpg](https://i.imgur.com/XKS1oqU.jpg)

Image taken from : https://github.com/fmfn/BayesianOptimization

## Notebook  Content
0. [Installing Bayesian global optimization library](#0) <br>    
1. [Loading the data](#1)
2. [Black box function to be optimized (LightGBM)](#2)
3. [Training LightGBM model](#3)
4. [Rank averaging](#4)
5. [Submission](#5)

<a id="0"></a> <br>
## 0. Installing Bayesian global optimization library

Let's install the latest release from pip

In [1]:
!pip install bayesian-optimization

<a id="1"></a> <br>
## 1. Loading the data

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from scipy.stats import rankdata
import lightgbm as lgb
from sklearn import metrics
import gc
import warnings

pd.set_option('display.max_columns', 200)

In [19]:
import pickle
f1=open(r'F:\zxd\Kaggle\santander-customer-transaction-prediction\Submit\train.txt','rb')
train_df=pickle.load(f1)
f1.close()
f=open(r'F:\zxd\Kaggle\santander-customer-transaction-prediction\Submit\test.txt','rb')
test_df=pickle.load(f)
f.close()

We are given anonymized dataset containing 200 numeric feature variables from var_0 to var_199. Let's have a look train dataset:

In [20]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,...,var_196_ratio,var_197_ratio,var_198_ratio,var_2_ratio,var_22_ratio,var_24_ratio,var_25_ratio,var_26_ratio,var_28_ratio,var_3_ratio,var_32_ratio,var_34_ratio,var_35_ratio,var_36_ratio,var_37_ratio,var_39_ratio,var_4_ratio,var_40_ratio,var_42_ratio,var_43_ratio,var_44_ratio,var_48_ratio,var_49_ratio,var_5_ratio,var_50_ratio,var_51_ratio,var_52_ratio,var_53_ratio,var_55_ratio,var_56_ratio,var_59_ratio,var_6_ratio,var_60_ratio,var_61_ratio,var_62_ratio,var_63_ratio,var_64_ratio,var_66_ratio,var_67_ratio,var_68_ratio,var_69_ratio,var_70_ratio,var_71_ratio,var_72_ratio,var_73_ratio,var_74_ratio,var_75_ratio,var_76_ratio,var_78_ratio,var_79_ratio,var_80_ratio,var_81_ratio,var_82_ratio,var_83_ratio,var_84_ratio,var_85_ratio,var_86_ratio,var_88_ratio,var_89_ratio,var_9_ratio,var_90_ratio,var_91_ratio,var_92_ratio,var_93_ratio,var_94_ratio,var_95_ratio,var_96_ratio,var_97_ratio,var_99_ratio,var_104_ratio,var_107_ratio,var_112_ratio,var_121_ratio,var_14_ratio,var_140_ratio,var_142_ratio,var_149_ratio,var_156_ratio,var_160_ratio,var_172_ratio,var_177_ratio,var_178_ratio,var_186_ratio,var_192_ratio,var_199_ratio,var_20_ratio,var_21_ratio,var_23_ratio,var_31_ratio,var_33_ratio,var_45_ratio,var_57_ratio,var_65_ratio,var_77_ratio,var_8_ratio,var_87_ratio,var_120_ratio,var_46_ratio,var_54_ratio,var_58_ratio
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,4.6667,3.8743,-5.2387,7.3746,11.5767,12.0446,11.6418,-7.0170,5.9226,-14.2136,16.0283,5.3253,12.9194,29.0460,-0.6940,5.1736,-0.7474,14.8322,11.2668,5.3822,2.0183,10.1166,16.1828,4.9590,2.0771,-0.2154,8.6748,9.5319,5.8056,22.4321,5.0109,-4.7010,21.6374,0.5663,5.1999,8.8600,43.1127,18.3816,-2.3440,23.4104,6.5199,12.1983,13.6468,13.8372,1.3675,2.9423,-4.5213,21.4669,9.3225,16.4597,7.9984,-1.7069,-21.4494,6.7806,11.0924,9.9913,14.8421,0.1812,8.9642,16.2572,...,1.218407,1.196663,0.812609,0.889169,0.934415,0.891793,0.975183,1.001917,1.072375,0.903033,0.871822,1.062310,1.090530,0.946887,1.147017,1.034529,0.994725,1.071756,1.037548,1.046615,1.015624,0.996981,0.973589,0.792603,0.999875,0.985295,0.933610,0.839139,1.040965,1.152663,0.844550,0.971426,1.015183,1.076094,0.982869,1.038455,1.023331,0.989312,1.027379,0.865785,0.899928,1.006357,0.947633,1.017684,0.765609,1.161926,1.004696,0.905047,1.036067,0.908603,1.137611,0.870138,1.061127,1.009080,0.917883,0.941532,1.041782,1.231490,0.696698,1.934524,0.967430,1.018207,1.088268,0.877328,1.120142,1.030191,0.912910,1.011907,1.018735,1.063925,1.015428,1.101427,1.083466,0.998691,1.055831,1.173750,1.023453,0.941332,1.194063,0.995053,0.868915,0.927647,0.964357,1.067237,0.999461,1.016186,1.035990,0.899440,1.026965,0.890931,1.006270,0.889903,0.919103,0.987538,0.960883,0.965914,0.942483,0.644645,0.928162,1.009814
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,10.8160,3.6783,-11.1147,1.8730,9.87

Test dataset:

In [21]:
test_df.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,...,var_196_ratio,var_197_ratio,var_198_ratio,var_2_ratio,var_22_ratio,var_24_ratio,var_25_ratio,var_26_ratio,var_28_ratio,var_3_ratio,var_32_ratio,var_34_ratio,var_35_ratio,var_36_ratio,var_37_ratio,var_39_ratio,var_4_ratio,var_40_ratio,var_42_ratio,var_43_ratio,var_44_ratio,var_48_ratio,var_49_ratio,var_5_ratio,var_50_ratio,var_51_ratio,var_52_ratio,var_53_ratio,var_55_ratio,var_56_ratio,var_59_ratio,var_6_ratio,var_60_ratio,var_61_ratio,var_62_ratio,var_63_ratio,var_64_ratio,var_66_ratio,var_67_ratio,var_68_ratio,var_69_ratio,var_70_ratio,var_71_ratio,var_72_ratio,var_73_ratio,var_74_ratio,var_75_ratio,var_76_ratio,var_78_ratio,var_79_ratio,var_80_ratio,var_81_ratio,var_82_ratio,var_83_ratio,var_84_ratio,var_85_ratio,var_86_ratio,var_88_ratio,var_89_ratio,var_9_ratio,var_90_ratio,var_91_ratio,var_92_ratio,var_93_ratio,var_94_ratio,var_95_ratio,var_96_ratio,var_97_ratio,var_99_ratio,var_104_ratio,var_107_ratio,var_112_ratio,var_121_ratio,var_14_ratio,var_140_ratio,var_142_ratio,var_149_ratio,var_156_ratio,var_160_ratio,var_172_ratio,var_177_ratio,var_178_ratio,var_186_ratio,var_192_ratio,var_199_ratio,var_20_ratio,var_21_ratio,var_23_ratio,var_31_ratio,var_33_ratio,var_45_ratio,var_57_ratio,var_65_ratio,var_77_ratio,var_8_ratio,var_87_ratio,var_120_ratio,var_46_ratio,var_54_ratio,var_58_ratio
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,-2.0248,-4.3554,13.9696,0.3458,7.5408,14.5001,7.7028,-19.0919,15.5806,16.1763,3.7088,18.8064,1.5899,3.0654,6.4509,14.1192,-9.4902,-2.1917,5.7107,3.7864,-1.7981,9.2645,2.0657,12.7753,11.3334,8.1462,-0.0610,3.5331,9.7804,8.7625,-15.6305,18.8766,11.2864,11.8362,13.3680,-31.9891,12.1776,8.7714,17.2011,16.8508,13.0534,14.4069,-4.8525,7.3213,-0.5259,16.6365,19.3036,6.4129,-5.3948,9.3269,11.9314,-3.5750,-0.7706,0.8705,6.9282,2.8914,5.9744,17.4851,5.0125,-1.4230,33.3401,0.8018,-4.7906,30.2708,26.8339,21.7205,7.3075,14.0810,3.1192,17.4265,9.4883,16.9060,14.5117,10.0276,-0.9706,20.4588,4.7945,20.4160,13.1633,7.9307,-7.6509,7.0834,15.2324,10.1416,5.9156,-0.5775,5.7600,30.3238,2.1251,...,1.071594,1.153006,1.027325,1.075693,0.929048,1.102077,1.084514,0.877153,1.015963,1.104166,1.087910,0.984106,1.113751,0.929617,0.923175,0.846942,1.036313,0.955849,0.997286,0.872239,0.991373,0.972361,0.986368,1.003991,1.000125,0.964512,0.953046,1.148224,1.027804,0.949831,0.900933,0.978014,1.022414,1.083533,1.011738,1.042202,0.967092,0.959436,1.025783,0.880721,1.113630,1.003394,1.055261,1.002495,1.027508,1.014519,0.927862,0.968576,1.071162,2.951772,0.988266,1.093841,1.541832,1.025807,1.008037,0.926751,0.924111,1.150322,1.141009,1.026816,1.036020,1.017468,0.957805,0.899621,0.592629,0.955423,0.884545,1.003864,0.981864,1.018372,0.986088,1.166043,1.004279,1.004569,2.899864,1.173750,0.969311,0.885866,1.057211,1.045474,1.312693,0.960050,1.022345,0.000000,0.876353,0.318066,0.972321,1.036051,1.001298,1.200054,0.945387,0.996193,1.002181,0.897869,1.086287,1.085689,1.135204,1.033577,0.921647,1.044576
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,-1.3809,-0.3310,14.1129,2.5667,5.4988,14.1853,7.0196,4.6564,29.1609,0.0910,12.1469,3.1389,5.2578,2.4228,16.2064,13.5023,-5.2341,-3.6648,5.7080,2.9965,-10.4720,11.4938,-0.9660,15.3445,10.6361,0.8966,6.7428,2.3421,12.8678,-1.5536,10.0309,3.13

Distribution of target variable

In [22]:
target = 'target'
predictors = train_df.columns.values.tolist()[2:]

In [23]:
train_df.target.value_counts()

0    179902
1     20098
Name: target, dtype: int64

The problem is unbalanced! 

In this kernel I will be using **50% Stratified rows** as holdout rows for the validation-set to get optimal parameters. Later I will use 5 fold cross validation in the final model fit.

In [24]:
bayesian_tr_index, bayesian_val_index  = list(StratifiedKFold(n_splits=2, shuffle=True, random_state=1).split(train_df, train_df.target.values))[0]

These `bayesian_tr_index` and `bayesian_val_index` indexes will be used for the bayesian optimization as training and validation index of training dataset.

<a id="2"></a> <br>
## 2. Black box function to be optimized (LightGBM)

As data is loaded, let's create the black box function for LightGBM to find parameters.

In [25]:
def LGB_bayesian(
    num_leaves,  # int
    min_data_in_leaf,  # int
    learning_rate,
    min_sum_hessian_in_leaf,    # int  
    feature_fraction,
    lambda_l1,
    lambda_l2,
    min_gain_to_split,
    max_depth):
    
    # LightGBM expects next three parameters need to be integer. So we make them integer
    num_leaves = int(num_leaves)
    min_data_in_leaf = int(min_data_in_leaf)
    max_depth = int(max_depth)

    assert type(num_leaves) == int
    assert type(min_data_in_leaf) == int
    assert type(max_depth) == int

    param = {
        'num_leaves': num_leaves,
        'max_bin': 63,
        'min_data_in_leaf': min_data_in_leaf,
        'learning_rate': learning_rate,
        'min_sum_hessian_in_leaf': min_sum_hessian_in_leaf,
        'bagging_fraction': 1.0,
        'bagging_freq': 5,
        'feature_fraction': feature_fraction,
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_gain_to_split': min_gain_to_split,
        'max_depth': max_depth,
        'save_binary': True, 
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,   

    }    
    
    
    xg_train = lgb.Dataset(train_df.iloc[bayesian_tr_index][predictors].values,
                           label=train_df.iloc[bayesian_tr_index][target].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )
    xg_valid = lgb.Dataset(train_df.iloc[bayesian_val_index][predictors].values,
                           label=train_df.iloc[bayesian_val_index][target].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )   

    num_round = 5000
    clf = lgb.train(param, xg_train, num_round, valid_sets = [xg_valid], verbose_eval=250, early_stopping_rounds = 50)
    
    predictions = clf.predict(train_df.iloc[bayesian_val_index][predictors].values, num_iteration=clf.best_iteration)   
    
    score = metrics.roc_auc_score(train_df.iloc[bayesian_val_index][target].values, predictions)
    
    return score

The above `LGB_bayesian` function will act as black box function for Bayesian optimization. I already defined the the trainng and validation dataset for LightGBM inside the `LGB_bayesian` function. 

The `LGB_bayesian` function takes values for `num_leaves`, `min_data_in_leaf`, `learning_rate`, `min_sum_hessian_in_leaf`, `feature_fraction`, `lambda_l1`, `lambda_l2`, `min_gain_to_split`, `max_depth` from Bayesian optimization framework. Keep in mind that `num_leaves`, `min_data_in_leaf`, and `max_depth` should be integer for LightGBM. But Bayesian Optimization sends continous vales to function. So I force them to be integer. I am only going to find optimal parameter values of them. The reader may increase or decrease number of parameters to optimize.

Now I need to give bounds for these parameters, so that Bayesian optimization only search inside the bounds.

In [26]:
# Bounded region of parameter space
bounds_LGB = {
    'num_leaves': (5, 20), 
    'min_data_in_leaf': (5, 20),  
    'learning_rate': (0.01, 0.3),
    'min_sum_hessian_in_leaf': (0.00001, 0.01),    
    'feature_fraction': (0.05, 0.5),
    'lambda_l1': (0, 5.0), 
    'lambda_l2': (0, 5.0), 
    'min_gain_to_split': (0, 1.0),
    'max_depth':(3,15),
}

Let's put all of them in BayesianOptimization object

In [27]:
from bayes_opt import BayesianOptimization

In [28]:
LGB_BO = BayesianOptimization(LGB_bayesian, bounds_LGB, random_state=13)

Now, let's the the key space (parameters) we are going to optimize:

In [29]:
print(LGB_BO.space.keys)

['num_leaves', 'min_data_in_leaf', 'learning_rate', 'min_sum_hessian_in_leaf', 'feature_fraction', 'lambda_l1', 'lambda_l2', 'min_gain_to_split', 'max_depth']


I have created the BayesianOptimization object (`LGB_BO`), it will not work until I call maximize. Before calling it, I want to explain two parameters of BayesianOptimization object (`LGB_BO`) which we can pass to maximize:
- `init_points`: How many initial random runs of **random** exploration we want to perform. In our case `LGB_bayesian` will be called `n_iter` times.
- `n_iter`: How many runs of bayesian optimization we want to perform after number of `init_points` runs. 

Now, it's time to call the function from Bayesian optimization framework to maximize. I allow `LGB_BO` object to run for 5 `init_points` (exploration) and 5 `n_iter` (exploitation).

In [30]:
init_points = 5
n_iter = 5

In [31]:
print('-' * 130)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)

----------------------------------------------------------------------------------------------------------------------------------
Initialization
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   feature_fraction |   lambda_l1 |   lambda_l2 |   learning_rate |   max_depth |   min_data_in_leaf |   min_gain_to_split |   min_sum_hessian_in_leaf |   num_leaves | 
Train until valid scores didn't improve in 50 rounds.
[250]	valid_0's auc: 0.832919
[500]	valid_0's auc: 0.860563
[750]	valid_0's auc: 0.87262
[1000]	valid_0's auc: 0.879457
[1250]	valid_0's auc: 0.883684
[1500]	valid_0's auc: 0.886434
[1750]	valid_0's auc: 0.888416
[2000]	valid_0's auc: 0.889621
[2250]	valid_0's auc: 0.890417
[2500]	valid_0's auc: 0.890861
[2750]	valid_0's auc: 0.891144
Early stopping, best iteration is:
[2750]	valid_0's auc: 0.891144
  

As the optimization is done, let's see what is the maximum value we have got.

In [34]:
LGB_BO.max

AttributeError: 'BayesianOptimization' object has no attribute 'max'

In [35]:
from bayes_opt import BayesianOptimization

In [36]:
# Bounded region of parameter space
pbounds = {'x': (2, 4), 'y': (-3, 3)}

In [38]:
def black_box_function(x, y):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -x ** 2 - (y - 1) ** 2 + 1

In [39]:
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

The validation AUC for parameters is 0.89 ! Let's see parameters is responsible for this score :)

In [40]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

Initialization
-----------------------------------------------------
 Step |   Time |      Value |         x |         y | 
    1 | 00m00s |  -23.02632 |    2.8340 |   -2.9993 | 
    2 | 00m00s |  -15.61668 |    3.4406 |   -1.1860 | 
Bayesian Optimization
-----------------------------------------------------
 Step |   Time |      Value |         x |         y | 
    3 | 00m00s |  -19.00000 |    4.0000 |    3.0000 | 
    4 | 00m01s |   -3.07171 |    2.0000 |    0.7322 | 
    5 | 00m01s |   -3.02674 |    2.0000 |    1.1635 | 


In [41]:
print(optimizer.max)

AttributeError: 'BayesianOptimization' object has no attribute 'max'

In [43]:
optimizer.probe(
    params={"x": 0.5, "y": 0.7},
    lazy=True,
)

AttributeError: 'BayesianOptimization' object has no attribute 'probe'

In [42]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	max
Iteration 1: 
	all


In [16]:
LGB_BO.max['params']

AttributeError: 'BayesianOptimization' object has no attribute 'max'

Now we can use these parameters to our final model!

Wait, I want to show one more cool option from BayesianOptimization library. You can probe the `LGB_bayesian` function, if you have an idea of the optimal parameters or it you get **parameters from other kernel** like mine [mine](https://www.kaggle.com/fayzur/customer-transaction-prediction). I will copy and paste parameters from my other kernel here. You can probe as folowing:

In [18]:
# parameters from version 2 of
#https://www.kaggle.com/fayzur/customer-transaction-prediction?scriptVersionId=10522231

LGB_BO.probe(
    params={'feature_fraction': 0.1403, 
            'lambda_l1': 4.218, 
            'lambda_l2': 1.734, 
            'learning_rate': 0.07, 
            'max_depth': 14, 
            'min_data_in_leaf': 17, 
            'min_gain_to_split': 0.1501, 
            'min_sum_hessian_in_leaf': 0.000446, 
            'num_leaves': 6},
    lazy=True, # 
)

OK, by default these will be explored lazily (lazy=True), meaning these points will be evaluated only the next time you call maximize. Let's do a maximize call of `LGB_BO` object.

In [19]:
LGB_BO.maximize(init_points=0, n_iter=0) # remember no init_points or n_iter

|   iter    |  target   | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_da... | min_ga... | min_su... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.86243
[500]	valid_0's auc: 0.880738
[750]	valid_0's auc: 0.887841
[1000]	valid_0's auc: 0.890905
[1250]	valid_0's auc: 0.892214
[1500]	valid_0's auc: 0.892655
Early stopping, best iteration is:
[1466]	valid_0's auc: 0.892672
|  11       |  0.8927   |  0.1403   |  4.218    |  1.734    |  0.07     |  14.0     |  17.0     |  0.1501   |  0.000446 |  6.0      |


Finally, the list of all parameters probed and their corresponding target values is available via the property LGB_BO.res.

In [20]:
for i, res in enumerate(LGB_BO.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	{'target': 0.8785456504868869, 'params': {'feature_fraction': 0.3999660847582191, 'lambda_l1': 1.1877061001745615, 'lambda_l2': 4.1213926633068425, 'learning_rate': 0.2900672674324699, 'max_depth': 14.67121336685872, 'min_data_in_leaf': 11.801738711259683, 'min_gain_to_split': 0.6090424627612779, 'min_sum_hessian_in_leaf': 0.007757509880902418, 'num_leaves': 14.624200171386038}}
Iteration 1: 
	{'target': 0.8915776403640969, 'params': {'feature_fraction': 0.3749082032826262, 'lambda_l1': 0.17518262050718658, 'lambda_l2': 1.492247354445897, 'learning_rate': 0.026968622645801674, 'max_depth': 13.284731311046386, 'min_data_in_leaf': 10.592810418122113, 'min_gain_to_split': 0.679847951578097, 'min_sum_hessian_in_leaf': 0.002570236693773035, 'num_leaves': 10.21371822728738}}
Iteration 2: 
	{'target': 0.8924701144136037, 'params': {'feature_fraction': 0.05423574653643624, 'lambda_l1': 1.7916689135248487, 'lambda_l2': 4.745470908391052, 'learning_rate': 0.07319071264818977, 'max

We have got a better validation score in the probe! As previously I ran `LGB_BO` only for 10 runs. In practice I increase it to arround 100.

In [21]:
LGB_BO.max['target']

0.8926723153666576

In [22]:
LGB_BO.max['params']

{'feature_fraction': 0.1403,
 'lambda_l1': 4.218,
 'lambda_l2': 1.734,
 'learning_rate': 0.07,
 'max_depth': 14.0,
 'min_data_in_leaf': 17.0,
 'min_gain_to_split': 0.1501,
 'min_sum_hessian_in_leaf': 0.000446,
 'num_leaves': 6.0}

Let's build a model together use therse parameters ;)

<a id="3"></a> <br>
## 3. Training LightGBM model

In [23]:
param_lgb = {
        'num_leaves': int(LGB_BO.max['params']['num_leaves']), # remember to int here
        'max_bin': 63,
        'min_data_in_leaf': int(LGB_BO.max['params']['min_data_in_leaf']), # remember to int here
        'learning_rate': LGB_BO.max['params']['learning_rate'],
        'min_sum_hessian_in_leaf': LGB_BO.max['params']['min_sum_hessian_in_leaf'],
        'bagging_fraction': 1.0, 
        'bagging_freq': 5, 
        'feature_fraction': LGB_BO.max['params']['feature_fraction'],
        'lambda_l1': LGB_BO.max['params']['lambda_l1'],
        'lambda_l2': LGB_BO.max['params']['lambda_l2'],
        'min_gain_to_split': LGB_BO.max['params']['min_gain_to_split'],
        'max_depth': int(LGB_BO.max['params']['max_depth']), # remember to int here
        'save_binary': True,
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,
    }

As you see, I assined `LGB_BO`'s optimal parameters to the `param_lgb` dictionary and they will be used to train a model with 5 fold.

Number of Kfolds:

In [24]:
nfold = 5

In [25]:
gc.collect()

159

In [26]:
skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=2019)

In [27]:
oof = np.zeros(len(train_df))
predictions = np.zeros((len(test_df),nfold))

i = 1
for train_index, valid_index in skf.split(train_df, train_df.target.values):
    print("\nfold {}".format(i))
    xg_train = lgb.Dataset(train_df.iloc[train_index][predictors].values,
                           label=train_df.iloc[train_index][target].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )
    xg_valid = lgb.Dataset(train_df.iloc[valid_index][predictors].values,
                           label=train_df.iloc[valid_index][target].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )   

    
    clf = lgb.train(param_lgb, xg_train, 5000, valid_sets = [xg_valid], verbose_eval=250, early_stopping_rounds = 50)
    oof[valid_index] = clf.predict(train_df.iloc[valid_index][predictors].values, num_iteration=clf.best_iteration) 
    
    predictions[:,i-1] += clf.predict(test_df[predictors], num_iteration=clf.best_iteration)
    i = i + 1

print("\n\nCV AUC: {:<0.2f}".format(metrics.roc_auc_score(train_df.target.values, oof)))


fold 1
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.865007
[500]	valid_0's auc: 0.883335
[750]	valid_0's auc: 0.890893
[1000]	valid_0's auc: 0.894616
[1250]	valid_0's auc: 0.896093
[1500]	valid_0's auc: 0.896619
Early stopping, best iteration is:
[1603]	valid_0's auc: 0.89686

fold 2
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.860373
[500]	valid_0's auc: 0.879811
[750]	valid_0's auc: 0.88715
[1000]	valid_0's auc: 0.891324
[1250]	valid_0's auc: 0.893482
[1500]	valid_0's auc: 0.894638
Early stopping, best iteration is:
[1545]	valid_0's auc: 0.894804

fold 3
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.86603
[500]	valid_0's auc: 0.884838
[750]	valid_0's auc: 0.892292
[1000]	valid_0's auc: 0.895874
[1250]	valid_0's auc: 0.897865
Early stopping, best iteration is:
[1310]	valid_0's auc: 0.898187

fold 4
Training until validation scores don't improve for 50 rounds.
[250]	v

So we got 0.90 AUC in 5 fold cross validation. And 5 fold prediction look like:

In [28]:
predictions

array([[0.41628346, 0.39960379, 0.38409074, 0.35182696, 0.40993432],
       [0.63407781, 0.6236449 , 0.61534627, 0.66678195, 0.66385472],
       [0.65491558, 0.7424437 , 0.63334601, 0.60578336, 0.70685663],
       ...,
       [0.05168246, 0.0319463 , 0.04241488, 0.03894831, 0.03609798],
       [0.41015456, 0.48430607, 0.53886698, 0.48510382, 0.49223725],
       [0.40877008, 0.39991497, 0.44468547, 0.48166165, 0.41461139]])

If you are still reading, bare with me. I will not take much of your time. :D We are almost done. Let's do a rank averaging on 5 fold predictions.

<a id="4"></a> <br>
## 4. Rank averaging

In [29]:
print("Rank averaging on", nfold, "fold predictions")
rank_predictions = np.zeros((predictions.shape[0],1))
for i in range(nfold):
    rank_predictions[:, 0] = np.add(rank_predictions[:, 0], rankdata(predictions[:, i].reshape(-1,1))/rank_predictions.shape[0]) 

rank_predictions /= nfold

Rank averaging on 5 fold predictions


Let's submit prediction to Kaggle.

<a id="5"></a> <br>
## 5. Submission

In [30]:
sub_df = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub_df["target"] = rank_predictions
sub_df[:10]

,ID_code,target
0,test_0,0.693134
1,test_1,0.875524
2,test_2,0.889322
3,test_3,0.881537
4,test_4,0.539898
5,test_5,0.012080
6,test_6,0.120680
7,test_7,0.849158
8,test_8,0.032887
9,test_9,0.125836


In [31]:
sub_df.to_csv("Customer_Transaction_rank_predictions.csv", index=False)

Do not forget to upvote :) Also fork and modify for your own use. ;)